In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import datetime as dt

import pandas as pd

from astropy.io import fits
from astropy.time import Time, TimeDelta

sys.path.append('/home/sandboxes/psalas/PTCS/AutoOOF/oof_au')

import scan_log
import SamplerData as sd

In [3]:
path = {"A":"/home/archive/science-data/",
        "T":"/home/archive/test-data/"}

df = pd.read_hdf("auto-oof-scans-15A_22A-surface_rms-13102022.hdf", "auto-oof")

In [4]:
df

,projid,oofmjd,scanmjd,oofscan,vscan,sscan,effscan,oofobj,effobj,surfrms,tant,tau0,oofrx
0,AGBT17A_440_01,58044.072419,58044.112755,3,9,10,11,0319+4130,1642+3948,308.943466,1.113642,0.1748,RcvrArray75_115
1,AGBT17A_440_01,58044.072419,58044.113206,3,9,10,12,0319+4130,1642+3948,282.817695,1.548866,0.1747,RcvrArray75_115
2,AGBT17A_440_01,58044.072419,58044.113669,3,9,10,13,0319+4130,1642+3948,313.043958,1.054656,0.1747,RcvrArray75_115
3,AGBT17A_440_01,58044.072419,58044.114120,3,9,10,14,0319+4130,1642+3948,328.670352,0.851502,0.1747,RcvrArray75_115
4,AGBT17A_440_01,58044.072419,58044.121528,3,15,16,17,0319+4130,1927+7358,NaN,0.622847,0.1742,RcvrArray75_115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,AGBT22A_044_01,59611.069329,59611.146933,3,111,112,114,0319+4130,0319+4130,5395.024781,0.979756,0.0116,RcvrArray75_115
2278,AGBT22A_044_01,59611.069329,59611.150012,3,115,116,117,0319+4130,0319+4130,483.834204,1.645936,0.0870,RcvrArray75_115
2279,AGBT22A_044_01,59611.069329,59611.150463,3,115,116,118,0319+4130,0319+4130,502.417089,1.297118,0.0870,RcvrArray75_115
2280,AGBT22A_044_01,59611.069329,59611.150926,3,115,116,119,0319+4130,0319+4130,548.486416,0.691426,0.0870,RcvrArray75_115


In [6]:
# Interface with the M&C samplers.
# This can take a few seconds.
w1sd = sd.SamplerData("Weather-Weather1-weather1", False)
w2sd = sd.SamplerData("Weather-Weather2-weather2", False)
w3sd = sd.SamplerData("Weather-Weather3-weather3", False)
stsd = sd.SamplerData("AntennaCharacterization-StructureTemperature-StructureTemp", False)

In [9]:
%%time

temp_w2_avg = []
temp_st_avg = []
wind_w1_avg = []
wind_w2_avg = []
wind_w3_avg = []

temp_w2_med = []
temp_w3_med = []
temp_st_med = []
wind_w1_med = []
wind_w2_med = []
wind_w3_med = []

temp_w2_std = []
temp_w3_std = []
temp_st_std = []
wind_w1_std = []
wind_w2_std = []
wind_w3_std = []

auto_oof_rx = []


idxf = len(df)

for idx in range(0,idxf):
    
    print(f"Index: {idx}")
    
    pid = df.at[idx, "projid"]
    

    projid = pid #"_".join(pid.split("_")[:-1])
    print(projid)
    project = "_".join(pid.split("_")[:-1])
    print(f"Project: {project}")
    session = pid.split("_")[2]
    print(f"Session: {session}")
    semester = pid.split("_")[0][-3:]
    print(f"Semester: {semester}")
    if pid[0] == "T":
        semester = f"20{semester[:-1]}"

    proj_path = f"{path[pid[0]]}/{semester}"
    print(proj_path)

    try:
        sl = scan_log.ScanLog(proj_path, projid)
    except FileNotFoundError:
        #proj_path = f"{path[pid[0]]}/{int(semester)+1}"
        #sl = scan_log.ScanLog(proj_path, projid)
        print("Scan log not found...")
        temp_w2_avg.append(np.nan)
        wind_w1_avg.append(np.nan)
        wind_w2_avg.append(np.nan)
        wind_w3_avg.append(np.nan)

        temp_w2_med.append(np.nan)
        wind_w1_med.append(np.nan)
        wind_w2_med.append(np.nan)
        wind_w3_med.append(np.nan)

        temp_w2_std.append(np.nan)
        wind_w1_std.append(np.nan)
        wind_w2_std.append(np.nan)
        wind_w3_std.append(np.nan)
        continue
        
    oof_scan = df.at[idx, "oofscan"]
    
    tbeg = sd.scanTime2tuple(sl.get_scan_dateobs(oof_scan))
    tend = sd.scanTime2tuple(sl.get_scan_dateobs(oof_scan+2))
    time_range = (tbeg, tend)
    
    # Now get the temperatures of the structure and from weather station 2.
    mjd_st, temps = sd.get_temp(stsd, time_range)
    mjd_w1, wind1 = sd.get_wind(w1sd, time_range, cols=(0,1))
    mjd_w2, wind2 = sd.get_wind(w2sd, time_range, cols=(0,1))
    mjd_w3, wind3 = sd.get_wind(w3sd, time_range, cols=(0,1))
    mjd_w2, temp1 = sd.get_wind(w2sd, time_range, cols=(0,5))
    _,      temp2 = sd.get_wind(w2sd, time_range, cols=(0,6))
    
    
    temp_w2_avg.append(0.5*(np.nanmean(temp1) + np.nanmean(temp2)))
    wind_w1_avg.append(np.nanmean(wind1))
    wind_w2_avg.append(np.nanmean(wind2))
    wind_w3_avg.append(np.nanmean(wind3))
    
    temp_w2_med.append(0.5*(np.nanmedian(temp1) + np.nanmedian(temp2)))
    wind_w1_med.append(np.nanmedian(wind1))
    wind_w2_med.append(np.nanmedian(wind2))
    wind_w3_med.append(np.nanmedian(wind3))
    
    temp_w2_std.append(0.5*(np.nanstd(temp1) + np.nanstd(temp2)))
    wind_w1_std.append(np.nanstd(wind1))
    wind_w2_std.append(np.nanstd(wind2))
    wind_w3_std.append(np.nanstd(wind3))
    

Index: 0
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 1
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 2
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 3
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 4
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 5
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 6
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 7
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 8
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/home/archive/science-data//17A
Index: 9
AGBT17A_440_01
Project: AGBT17A_440
Session: 01
Semester: 17A
/h

Index: 79
AGBT17B_044_01
Project: AGBT17B_044
Session: 01
Semester: 17B
/home/archive/science-data//17B
Index: 80
AGBT17B_044_01
Project: AGBT17B_044
Session: 01
Semester: 17B
/home/archive/science-data//17B
Index: 81
AGBT17B_044_01
Project: AGBT17B_044
Session: 01
Semester: 17B
/home/archive/science-data//17B
Index: 82
AGBT17B_044_01
Project: AGBT17B_044
Session: 01
Semester: 17B
/home/archive/science-data//17B
Index: 83
AGBT17B_044_01
Project: AGBT17B_044
Session: 01
Semester: 17B
/home/archive/science-data//17B
Index: 84
AGBT17B_044_01
Project: AGBT17B_044
Session: 01
Semester: 17B
/home/archive/science-data//17B
Index: 85
AGBT17B_044_02
Project: AGBT17B_044
Session: 02
Semester: 17B
/home/archive/science-data//17B
Index: 86
AGBT17B_044_02
Project: AGBT17B_044
Session: 02
Semester: 17B
/home/archive/science-data//17B
Index: 87
AGBT17B_044_02
Project: AGBT17B_044
Session: 02
Semester: 17B
/home/archive/science-data//17B
Index: 88
AGBT17B_044_02
Project: AGBT17B_044
Session: 02
Semest

Index: 158
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 159
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 160
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 161
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 162
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 163
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 164
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 165
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 166
AGBT17B_044_04
Project: AGBT17B_044
Session: 04
Semester: 17B
/home/archive/science-data//17B
Index: 167
AGBT17B_044_04
Project: AGBT17B_044
Session:

Index: 237
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 238
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 239
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 240
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 241
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 242
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 243
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 244
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 245
AGBT17B_044_05
Project: AGBT17B_044
Session: 05
Semester: 17B
/home/archive/science-data//17B
Index: 246
AGBT17B_044_05
Project: AGBT17B_044
Session:

Index: 316
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 317
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 318
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 319
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 320
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 321
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 322
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 323
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 324
AGBT17B_044_08
Project: AGBT17B_044
Session: 08
Semester: 17B
/home/archive/science-data//17B
Index: 325
AGBT17B_044_08
Project: AGBT17B_044
Session:

...

Index: 1931
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1932
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1933
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1934
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1935
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1936
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1937
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1938
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1939
AGBT20B_451_02
Project: AGBT20B_451
Session: 02
Semester: 20B
/home/archive/science-data//20B
Index: 1940
AGBT20B_451_02
Project: AGBT20B_45

Index: 2009
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2010
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2011
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2012
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2013
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2014
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2015
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2016
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2017
AGBT21B_228_03
Project: AGBT21B_228
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2018
AGBT21B_228_03
Project: AGBT21B_22

Index: 2087
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2088
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2089
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2090
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2091
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2092
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2093
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2094
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2095
AGBT21B_024_12
Project: AGBT21B_024
Session: 12
Semester: 21B
/home/archive/science-data//21B
Index: 2096
AGBT21B_024_12
Project: AGBT21B_02

Index: 2165
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2166
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2167
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2168
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2169
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2170
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2171
AGBT21B_024_19
Project: AGBT21B_024
Session: 19
Semester: 21B
/home/archive/science-data//21B
Index: 2172
AGBT21B_024_21
Project: AGBT21B_024
Session: 21
Semester: 21B
/home/archive/science-data//21B
Index: 2173
AGBT21B_024_21
Project: AGBT21B_024
Session: 21
Semester: 21B
/home/archive/science-data//21B
Index: 2174
AGBT21B_024_21
Project: AGBT21B_02

Index: 2243
AGBT21B_065_03
Project: AGBT21B_065
Session: 03
Semester: 21B
/home/archive/science-data//21B
Index: 2244
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2245
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2246
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2247
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2248
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2249
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2250
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2251
AGBT21B_065_04
Project: AGBT21B_065
Session: 04
Semester: 21B
/home/archive/science-data//21B
Index: 2252
AGBT21B_065_04
Project: AGBT21B_06

In [10]:
df["weather1_wind_avg"] = wind_w1_avg
df["weather1_wind_med"] = wind_w1_med
df["weather1_wind_std"] = wind_w1_std

In [11]:
df["weather2_wind_avg"] = wind_w2_avg
df["weather2_wind_med"] = wind_w2_med
df["weather2_wind_std"] = wind_w2_std

In [12]:
df["weather3_wind_avg"] = wind_w3_avg
df["weather3_wind_med"] = wind_w3_med
df["weather3_wind_std"] = wind_w3_std

In [13]:
df["weather2_temp_avg"] = temp_w2_avg
df["weather2_temp_med"] = temp_w2_med
df["weather2_temp_std"] = temp_w2_std

In [16]:
df.to_hdf("auto-oof-scans-15A_22A-Argus-14102022.hdf", "auto-oof")

/home/sandboxes/psalas/PTCS/AutoOOF/psalas_venv/lib/python3.7/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'auto-oof'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [19]:
df.to_csv("auto-oof-scans-15A_22A-Argus-14102022.csv", index=False)

In [15]:
df.columns

Index(['projid', 'oofmjd', 'scanmjd', 'oofscan', 'vscan', 'sscan', 'effscan',
       'oofobj', 'effobj', 'surfrms', 'tant', 'tau0', 'oofrx',
       'weather1_wind_avg', 'weather1_wind_med', 'weather1_wind_std',
       'weather2_wind_avg', 'weather2_wind_med', 'weather2_wind_std',
       'weather3_wind_avg', 'weather3_wind_med', 'weather3_wind_std',
       'weather2_temp_avg', 'weather2_temp_med', 'weather2_temp_std'],
      dtype='object')